# **1. Import Library**

In [8]:
# Standard Libraries
import os
import time
import random
from tqdm import tqdm # type: ignore
import pickle
from collections import Counter
import asyncio
import aiohttp # type: ignore

# Seed Setup
import numpy as np
np.random.seed(0)  # Set seed sebelum pengacakan

# Data Manipulation & scrapping
import requests
import pandas as pd
from bs4 import BeautifulSoup

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

# Pandas Configuration
pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)
pd.set_option("display.float_format", lambda x: "%.4f" % x)

# Warnings Configuration
import warnings
warnings.filterwarnings("ignore")

# Setup logging configuration
import logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger()

# **2. Load Dataset**

In [9]:
mainPath =  'C:/Users/rendi/ITTS DATA SCIENCE/Semester 8/Laskar AI - Dicoding x NVIDIA/Capstone Project/Capstone - Course Recommender Systems/' # Path of the main project
dataPath = os.path.join(mainPath, 'Dataset/') # Path of the dataset

In [10]:
# Load the CSV file from the URL
df_courses = pd.read_csv(dataPath + 'udemy_courses_new.csv')
df_interactions = pd.read_csv(dataPath + 'synthetic_user_interactions.csv')
df_courses.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3672 entries, 0 to 3671
Data columns (total 14 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   course_id            3672 non-null   int64  
 1   course_title         3672 non-null   object 
 2   url                  3672 non-null   object 
 3   is_paid              3672 non-null   bool   
 4   price                3672 non-null   int64  
 5   num_subscribers      3672 non-null   int64  
 6   num_reviews          3672 non-null   int64  
 7   num_lectures         3672 non-null   int64  
 8   level                3672 non-null   object 
 9   content_duration     3672 non-null   float64
 10  published_timestamp  3672 non-null   object 
 11  subject              3672 non-null   object 
 12  total_interactions   3672 non-null   int64  
 13  total_users          3672 non-null   int64  
dtypes: bool(1), float64(1), int64(7), object(5)
memory usage: 376.7+ KB


In [29]:
links = df_courses[["course_title", "url"]]

In [32]:
# Fungsi untuk mendapatkan gambar dari URL kursus
def get_image_url(course_url):
    try:
        response = requests.get(course_url)
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')
            img_tag = soup.find('span', class_='intro-asset--img-aspect--3gluH').find('img')
            return img_tag['src']
        else:
            return None
    except Exception as e:
        # print(f"Error retrieving image for {course_url}: {e}")
        return None

# Menambahkan kolom "image_banner_url" ke DataFrame dengan iterasi menggunakan tqdm untuk progress bar
start_time = time.time()  # Start timing the process

# Menggunakan tqdm untuk menunjukkan progress bar
df_courses['image_banner_url'] = [
    get_image_url(url) for url in tqdm(df_courses['url'], desc="Processing Courses", unit="course")
]

# Menghitung waktu eksekusi
end_time = time.time()
execution_time = end_time - start_time

# Menampilkan DataFrame dan waktu eksekusi
display(df_courses)
print(f"Execution Time: {execution_time:.2f} seconds")

KeyboardInterrupt: 

In [ ]:
# # Fungsi untuk mendapatkan gambar dari URL kursus
# def get_image_url(course_url):
#     try:
#         response = requests.get(course_url)
#         if response.status_code == 200:
#             soup = BeautifulSoup(response.content, 'html.parser')
#             img_tag = soup.find('span', class_='intro-asset--img-aspect--3gluH').find('img')
#             return img_tag['src']
#         else:
#             return None
#     except Exception as e:
#         print(f"Error retrieving image for {course_url}: {e}")
#         return None

In [ ]:
# Menambahkan kolom "image_banner_url" ke DataFrame dengan iterasi
df_courses['image_banner_url'] = df_courses['url'].apply(get_image_url)

Error retrieving image for https://www.udemy.com/trading-penny-stocks-a-guide-for-all-levels/: 'NoneType' object has no attribute 'find'
Error retrieving image for https://www.udemy.com/day-trading-stock-options-3/: 'NoneType' object has no attribute 'find'
Error retrieving image for https://www.udemy.com/trading-options-using-money-flow/: 'NoneType' object has no attribute 'find'
Error retrieving image for https://www.udemy.com/hedge-fund-strategy-trading-with-sentiment-analysis/: 'NoneType' object has no attribute 'find'
Error retrieving image for https://www.udemy.com/learn-basic-technical-analysis/: 'NoneType' object has no attribute 'find'
Error retrieving image for https://www.udemy.com/intro-to-financial-statements-for-entrepreneurs/: 'NoneType' object has no attribute 'find'
Error retrieving image for https://www.udemy.com/forexmacross/: 'NoneType' object has no attribute 'find'
Error retrieving image for https://www.udemy.com/launch-your-amazing-forex-robot-in-30-minutes-no-co

In [ ]:
df_courses[["course_title", "url"]].head()

,course_title,url
0,Ultimate Investment Banking Course,https://www.udemy.com/ultimate-investment-bank...
1,Complete GST Course & Certification - Grow You...,https://www.udemy.com/goods-and-services-tax/
2,Financial Modeling for Business Analysts and C...,https://www.udemy.com/financial-modeling-for-b...
3,Beginner to Pro - Financial Analysis in Excel ...,https://www.udemy.com/complete-excel-finance-c...
4,How To Maximize Your Profits Trading Options,https://www.udemy.com/how-to-maximize-your-pro...


# **4.  Save Scraping Result to Dataframe**

In [ ]:
df_courses.to_csv(dataPath + 'udemy_courses_with_images.csv', index=False)

# **5. Load df with URL Img**

In [11]:
# Load the CSV file from the URL
df_with_url_img = pd.read_csv(dataPath + '[Before_Fillna] udemy_courses_with_images.csv')
df_with_url_img.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3672 entries, 0 to 3671
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   course_id            3672 non-null   int64  
 1   course_title         3672 non-null   object 
 2   url                  3672 non-null   object 
 3   is_paid              3672 non-null   bool   
 4   price                3672 non-null   int64  
 5   num_subscribers      3672 non-null   int64  
 6   num_reviews          3672 non-null   int64  
 7   num_lectures         3672 non-null   int64  
 8   level                3672 non-null   object 
 9   content_duration     3672 non-null   float64
 10  published_timestamp  3672 non-null   object 
 11  subject              3672 non-null   object 
 12  total_interactions   3672 non-null   int64  
 13  total_users          3672 non-null   int64  
 14  image_banner_url     2063 non-null   object 
dtypes: bool(1), float64(1), int64(7), obje

In [12]:
df_with_url_img["image_banner_url"].isnull().sum()

np.int64(1609)

In [ ]:
df_with_url_img["image_banner_url"].fillna("https://www.udemy.com/staticx/udemy/images/v6/default-meta-image.png", inplace=True)

In [ ]:
df_with_url_img.to_csv(dataPath + '[After_Fillna] udemy_courses_with_images.csv', index=False)